In [1]:
##  
import math
import numpy as np
import os
import pandas as pd

# Annotate 
def GetDist2Feats(x,H):
    print(H+".strand", x.shape)
    if x[H+".strand"] == "+":
        return x["str.start"] - x[H+'.start']
    elif x[H+".strand"] == "-":
        return -1*(x["str.start"] - x[H+'.stop'])
    else:
        return float(None)
    
def GetDist2Feat(strand, X, start,end):
   
    if str(strand) == "+":
        return X['best.str.start'].values[0] - start
    elif str(strand) == "-":
        return -1*(X['best.str.start'].values[0] - end)
    else:
        return float('NaN')   
    
def AnnotateFeatures(X, ANOT):
    ANOT['start']=ANOT['gene.start'] ; ANOT['stop']=ANOT['gene.stop']
    chrom=['chr'+str(x) for x in range(1,23,1) ]+['X','Y']
    Annotated=[]
    
    for ch in chrom:
        Feature = X.loc[X['chrom']==ch]
        Names = ANOT.loc[ANOT['gene.chr']==ch].head(50)
        starts = list(Feature['start'])
        stops = list(Feature['stop'])
        Genes=[]
        i=-1
        for start in starts:
            i=i+1
            stop=stops[i]
            gene = Names.loc[(Names['start']<=start) & (Names['stop']>=stop)]['gene.id']
            print(start, stop, gene)
            try:
                Genes.append(list(gene)[0])
            except:
                Genes.append('nan')
        print(ch, ' AnnotateFeatures', Names.shape, len(Genes))      
        Feature['gene']= Genes  
        Annotated.append(Feature)
        break
    OUT = pd.concat(Annotated)
    return OUT

def ISINTHISFEATURE(X, feature):           #X=STRS
    start = X["start"].values[0]
    end = X["stop"].values[0]
    F_strs = feature[(feature["start"] <= start) & (feature["stop"] >= start)]
    if F_strs.shape[0] == 0:
        return 0,'nan', 'nan', 'nan'
    else:
        return 1,F_strs["start"].values[0], F_strs["stop"].values[0], F_strs["strand"].values[0]   

ch = [i+1 for i in range(22)]#+['X','Y'] 
data=0; OUT=0;OUT1=0; p=0;i=0

In [2]:
DATA = '/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/SUMMARY/Merged_Causality_best_tissue.table'
ANOT = '/home/szfeupe/projects/GTEX_eSTRs/data/Lin_Reg/Gene_Exp_Annotation.txt'
UTR3 = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/3UTR_table'
UTR5 = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/5UTR_table'
EXON = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/Exons_table'
ITRN = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/Introns_table'
CO_EX= '/storage/szfeupe/Runs/GTEx_estr/FEATURES/CodingExons_table'
MOTIF= '/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed'
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')


FEATCOL  = ['chrom', 'start', 'stop','description','score','strand', 'accession.code']


data0 = pd.read_csv(DATA, sep='\t')
data0['best.str.start']=data0['best.str.start'].astype(float)
data = pd.merge(data0, Gene_table[["gene","chrom","gene.name"]], on=["chrom","gene"])


anot = pd.read_csv(ANOT,sep=',')                   ;  anot['gene']=anot['gene.id']

utr3 = pd.read_csv(UTR3, sep='\t', header=None)
utr3['accession.code']=[x.split('_ut')[0] for x in list(utr3[3])] ;  utr3.columns=FEATCOL

utr5 = pd.read_csv(UTR5, sep='\t', header=None)    
utr5['accession.code']=[x.split('_ut')[0] for x in list(utr5[3])] ;  utr5.columns=FEATCOL

exon = pd.read_csv(EXON , sep='\t', header=None)   
exon['accession.code']=[x.split('_ex')[0] for x in list(exon[3])] ;  exon.columns=FEATCOL

itrn = pd.read_csv(ITRN, sep='\t', header=None)    
itrn['accession.code']=[x.split('_in')[0] for x in list(itrn[3])] ;  itrn.columns=FEATCOL

coex = pd.read_csv(CO_EX, sep='\t', header=None)   
coex['accession.code']=[x.split('_cd')[0] for x in list(coex[3])] ;  coex.columns=FEATCOL

moti = pd.read_csv(MOTIF, sep='\t', header=None)
moti.columns=['chrom', 'start', 'stop', 'motif_len', 'motif']
moti['str.start']=moti['start'].astype(float)


In [3]:
data['str.start']=data['best.str.start']
O = pd.merge(data, anot[["gene","gene.start","gene.stop","gene.strand"]], on=["gene"])

OUT1 = pd.merge(O, moti, on=["chrom","str.start"])
features = [utr3, utr5, exon, itrn]
headers = ['UTR3', 'UTR5', 'EXON', 'INTRON']
p=0
for feature in features:
    HEAD = headers[p]
    print(HEAD)
    OUT = OUT1
    frames = []
    print(p, OUT1.shape, HEAD)
    for C in ch:
        print(C)
        DF=[]
        starts = []
        ends = []
        Dist2Feat = []
        ds = OUT.loc[OUT['chrom']=='chr'+str(C)]
        ftr = feature.loc[feature['chrom']=='chr'+str(C)]
        ftr[HEAD+'.start']=ftr['start']
        ftr[HEAD+'.stop'] = ftr['stop']
        ftr[HEAD+'.strand'] = ftr['strand']
        for i in range(ds.shape[0]):
            X = ds.loc[[ds.index.values[i]]]
            output, output1, output2, strands = ISINTHISFEATURE(X, ftr)
            DF.append(output)
            starts.append(output1)
            ends.append(output2)
            if output==0:
                Dist2Feat.append('NaN')
            else:
                Dist2Feat.append(GetDist2Feat(strands, X, output1,output2))
        ds[HEAD]=DF
        ds[HEAD+'.start']=starts
        ds[HEAD+'.stop']=ends
        ds[HEAD+'.strand']=ftr['strand']
        ds['dist.to.'+HEAD]=Dist2Feat
        frames.append(ds)
    OUT1 = pd.concat(frames)
    p=p+1
    
#OUT1.to_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/SUMMARY/Merged_Causality_Features.table', sep='\t', index=None)
OUT1

UTR3
0 (11963, 15) UTR3
1


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/p

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
UTR5
1 (11963, 20) UTR5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
EXON
2 (11963, 25) EXON
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
INTRON
3 (11963, 30) INTRON
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


,gene,chrom,best.str.start,best.score,best.tissue,best.q,gene.name,str.start,gene.start,gene.stop,...,EXON,EXON.start,EXON.stop,EXON.strand,dist.to.EXON,INTRON,INTRON.start,INTRON.stop,INTRON.strand,dist.to.INTRON
3,ENSG00000060642.6,chr1,27190822.0,0.013722,Artery-Tibial,0.411755,PIGV,27190822.0,27113963,27124889,...,1,27189632,27190947,+,1190,0,nan,nan,+,NaN
19,ENSG00000117500.8,chr1,93717553.0,0.016377,Muscle-Skeletal,0.954101,TMED5,93717553.0,93615299,93646285,...,0,nan,nan,+,NaN,1,93712511,93720027,+,5042
26,ENSG00000116783.10,chr1,74933940.0,0.019539,Artery-Tibial,0.869954,TNNI3K,74933940.0,74663947,75010112,...,0,nan,nan,+,NaN,1,74929234,74954872,+,4706
37,ENSG00000116668.8,chr1,185238499.0,0.043199,Lung,0.578780,SWT1,185238499.0,185126212,185260897,...,0,nan,nan,+,NaN,1,185200840,185240454,+,37659
38,ENSG00000116679.11,chr1,185238499.0,0.007637,Muscle-Skeletal,1.000000,IVNS1ABP,185238499.0,185265520,185286461,...,0,nan,nan,+,NaN,1,185200840,185240454,+,37659
40,ENSG00000269113.3,chr1,48194556.0,0.023123,Esophagus-Mucosa,0.847674,TRABD2B,48194556.0,48226200,48462567,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
51,ENSG00000174567.7,chr1,204245761.0,0.031929,WholeBlood,0.526899,GOLT1A,204245761.0,204167288,204183220,...,0,nan,nan,+,NaN,1,204243937,204328821,+,83060
66,ENSG00000158966.9,chr1,64943268.0,0.113584,Artery-Tibial,0.226453,CACHD1,64943268.0,64936428,65158741,...,0,nan,nan,+,NaN,1,64936625,65016276,+,6643
120,ENSG00000213625.4,chr1,65882749.0,0.016765,Lung,0.620958,LEPROT,65882749.0,65886270,65901690,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
131,ENSG00000121644.14,chr1,244904059.0,0.021602,WholeBlood,0.557806,DESI2,244904059.0,244816237,244872335,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN


In [7]:
OUT1.to_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/SUMMARY/Merged_Causality_Features.table', sep='\t', index=None)
data=OUT1
#data=pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/SUMMARY/Merged_Causality_Features.table', sep='\t')
genes=['ENSG00000164346.5',
 'ENSG00000186470.9',
 'ENSG00000204520.8',
 'ENSG00000188234.9',
 'ENSG00000149084.7',
 'ENSG00000166435.11',
 'ENSG00000166896.3',
 'ENSG00000090857.9',
 'ENSG00000141698.12',
 'ENSG00000006282.15',
 'ENSG00000174652.13']

data.loc[data["gene"].isin(genes)][["gene","gene.name","motif","motif_len","chrom","str.start","best.score","best.tissue","EXON","INTRON","UTR3","UTR5"]]


,gene,gene.name,motif,motif_len,chrom,str.start,best.score,best.tissue,EXON,INTRON,UTR3,UTR5
8359,ENSG00000164346.5,NSA2,AT,2,chr5,74013303.0,7.707686e-06,Esophagus-Mucosa,0,1,0,0
5506,ENSG00000186470.9,BTN3A2,N,6,chr6,26360419.0,4.786874e-02,Artery-Tibial,0,0,0,0
7430,ENSG00000188234.9,AGAP4,AAAC,4,chr10,46292054.0,2.586107e-02,Muscle-Skeletal,0,0,0,0
2026,ENSG00000166435.11,XRRA1,ACAG,4,chr11,74710172.0,4.354364e-01,Muscle-Skeletal,0,1,0,0
8916,ENSG00000149084.7,HSD17B12,AT,2,chr11,43793574.0,2.857083e-02,Artery-Tibial,0,1,0,0
331,ENSG00000166896.3,XRCC6BP1,AT,2,chr12,58306637.0,1.684661e-03,Esophagus-Mucosa,0,0,0,0
10180,ENSG00000090857.9,PDPR,AAGGG,5,chr16,70182468.0,7.726115e-01,Artery-Tibial,0,1,0,0
7547,ENSG00000006282.15,SPATA20,AAAG,4,chr17,48622244.0,8.914742e-07,Cells-Transformedfibroblasts,0,0,0,0
10793,ENSG00000141698.12,NT5C3B,AAAG,4,chr17,39982464.0,3.641446e-01,Artery-Tibial,0,1,0,0


In [1]:
Top10=[]
for T in tissu:
    M = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/'+T+'/Master.table', sep='\t')
    Top = M.sort_values('causality.score',ascending = False).head(10)
    Top10.append(Top)
Top = pd.concat(Top10)
Top

gene=['ENSG00000066427.17']

#Get the epression for gene from all tissues
for tissue in tissu:
#    MT = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/'+tissue+'/Master.table', sep='\t')
    MT = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/'+tissue+'/HH/All_genesh2_gcta',sep='\t')
    MT = MT.loc[MT['gene'].isin(gene)]
    data.append(MT)
    #print(len(data) )  #len(data))  
result = pd.concat(data, axis=0, )
#result.columns=pd.Series(tissu)
#Save into file
#result.to_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/'+gene+'Expression.txt', sep='\t')
gene=gene[0]
result

NameError: name 'tissu' is not defined

In [ ]:
#Making genecode table with features
HCOL=['gene','chrom','gene.name','feature.type','start','stop','G.strand','gene.type','database','score','genomic.phase']

annot = pd.read_csv('/storage/szfeupe/data/gencode.v19.annotation.gtf.gz', sep='\t', skiprows=5, header=None)
IDs= annot[8].str[1:-1].str.split('"', expand=True)
del annot[8]
annot.columns=['chrom','database','feature.type','start','stop','score','G.strand','genomic.phase']
annot['gene']=IDs[1]
annot['gene.name']=IDs[9]
annot['gene.type']=IDs[5]
annot = annot[HCOL]



In [25]:
Op=OUT2[['gene', 'chrom', 'best.str.start', 'gene.start', 'gene.stop','motif_len', 'motif', 'UTR3', 'UTR3.start','UTR3.stop', 'dist.to.UTR3']]
Op.loc[Op["UTR3"]==1]
#     , 'UTR5', 'UTR5.start','UTR5.stop','dist.to.UTR5', 'EXON', 'EXON.start',
#       'EXON.stop','dist.to.EXON', 'INTRON', 'INTRON.start',
#       'INTRON.stop', 'INTRON.strand', 'dist.to.INTRON', 'dist.to.tss',
#       'dist.to.tes']

,gene,chrom,best.str.start,gene.start,gene.stop,motif_len,motif,UTR3,UTR3.start,UTR3.stop,dist.to.UTR3
0,ENSG00000060642.6,chr1,27190822.0,27113963,27124889,2,AC,1,27190450,27190947,372
30,ENSG00000203684.5,chr1,228369874.0,228351787,228353213,4,AATG,1,228363214,228369958,6660
31,ENSG00000143761.9,chr1,228369874.0,228270361,228286912,4,AATG,1,228363214,228369958,6660
187,ENSG00000163467.7,chr1,156262168.0,156307105,156316786,2,AC,1,156261389,156262234,779
188,ENSG00000189030.8,chr1,156262168.0,156268415,156269428,2,AC,1,156261389,156262234,779
189,ENSG00000160781.11,chr1,156262168.0,156213206,156217881,2,AC,1,156261389,156262234,779
190,ENSG00000163472.14,chr1,156262168.0,156252726,156262976,2,AC,1,156261389,156262234,779
196,ENSG00000134686.12,chr1,33981918.0,33789224,33896653,2,AC,1,33979598,33982170,252
201,ENSG00000034971.10,chr1,171562388.0,171604557,171621823,2,AT,1,171560986,171562650,1402
347,ENSG00000018625.10,chr1,160009164.0,160085549,160113381,2,AC,1,160007256,160011182,2018


In [29]:
motifs = [(x, list(OUT2["motif"]).count(x) ) for x in set(list(OUT2["motif"]))]
#len(motifs)
[y for y in motifs if y[1]>=250]

[('N', 1137),
 ('AAC', 263),
 ('AG', 469),
 ('AAAAT', 300),
 ('AAAC', 528),
 ('AAAT', 1398),
 ('AC', 4068),
 ('AAAAC', 315),
 ('AT', 1123),
 ('AAT', 579)]

In [ ]:
('AAT', 579)
('AT', 1123)
('AGAT', 211)
('AAAAC', 315)
('AC', 4068)
('AAAG', 209)
('AAAT', 1398)

